In [59]:
import ee
import os
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from geemap import png_to_gif
from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
%matplotlib inline

In [115]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -5.488014221191406,
              36.21103919432822
            ],
            [
              -5.4931640625,
              36.0477110422792
            ],
            [
              -5.303306579589844,
              36.047155868712984
            ],
            [
              -5.3015899658203125,
              36.214086231406995
            ],
            [
              -5.488014221191406,
              36.21103919432822
            ]
          ]
        ]
      }
    }
  ]
}

In [116]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

start_date = "2021-12-01" # 
end_date = "2021-12-31" #

In [120]:
ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

In [121]:
import time
acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/01/21',
 '12/02/21',
 '12/02/21',
 '12/02/21',
 '12/03/21',
 '12/07/21',
 '12/08/21',
 '12/08/21',
 '12/09/21',
 '12/13/21',
 '12/14/21',
 '12/14/21',
 '12/14/21',
 '12/15/21',
 '12/19/21',
 '12/20/21',
 '12/20/21',
 '12/21/21',
 '12/25/21',
 '12/26/21',
 '12/26/21']

In [133]:
im_list = ffa_db.toList(ffa_db.size())
index = 3
url = ee.Image(im_list.get(index)).select('VH').clip(aoi).getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=1200)

In [132]:
ee.Image(im_list.get(index)).getInfo()

{'type': 'Image',
 'bands': [{'id': 'VV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [29098, 21781],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 184425.58254112263, 0, -10, 4172225.2942469036]},
  {'id': 'VH',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [29098, 21781],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 184425.58254112263, 0, -10, 4172225.2942469036]},
  {'id': 'angle',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [21, 10],
   'crs': 'EPSG:32630',
   'crs_transform': [12899.982031659398,
    -3250.878542552062,
    217131.24200735777,
    2047.8700559330173,
    20161.766713714693,
    3958880.680289361]}],
 'id': 'COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20211202T181754_20211202T181819_029850_03903A_0808',
 'version': 1643800568859737,
 'properties': {'GRD_Post_Processing_start': 1638471174842,
  'sliceNumber': 4,
  'GRD_Post_Processing_facility_name': 'Copernicus

In [135]:
import requests

img_data = requests.get(url).content
with open('image_name2.png', 'wb') as handler:
    handler.write(img_data)